In [55]:
import os
import requests
import csv
import numpy as np
import pandas as pd

In [2]:
#change these
filename = "21-202rw"
html_ = f"{filename}.html"
csv_ =  f"{filename}.csv"
meet = "2021 PV Maryland State LC Championship - 6/4/2021 to 6/6/2021"

In [3]:
# Path
data_folder = "data"
 
# Join various path components
html_file = os.path.join(data_folder, html_)
print(html_file)

data\21-202rw.html


In [4]:
#Creating an output file in writing mode
csv_file = os.path.join(data_folder,csv_)

#csv header
header = ["meet", "event", "rank", "name", "age", "team", "seed", "finals"]
#assign 'delete' event for delition later
event = "delete"

with open(csv_file, 'w', encoding='UTF8') as f:
    #create a writer
    writer = csv.writer(f, delimiter = "|")
    # write the header
    writer.writerow(header)
    #iterate html
    with open(html_file, "r") as line:
        line_string = line.readlines()
        #this will assign event column
        for each_line in line_string:
            if each_line[0:2] == "<b":
                event = each_line.replace("<b>", "")\
                                 .replace("</b>", "")\
                                 .replace("&amp;", "&")\
                                 .replace("\n", " ")\
                                 .rstrip()
            else:
                #assign each column and strip right spaces
                rank = each_line[0:4].replace(" ", "").replace("=", "").rstrip()
                name = each_line[4:29].rstrip()
                age = each_line[28:33].replace(" ","").rstrip()
                team = each_line[33:50].replace(" ","").rstrip()
                seed = each_line[51:65].replace(" ","").rstrip()
                finals = each_line[61:80].replace(" ","").rstrip()
                #this will not write uncessary rows
                if rank != "" and event != 'delete'\
                   and seed != "" and finals != "":
                    writer.writerow([meet, event, rank, name, age, team, seed, finals])



In [5]:
#read csv to data frame
df = pd.read_csv(csv_file, sep="|"  , engine="python")

In [46]:
#filtering examples
filt = (df["name"] == "Kertajaya, Evelyn") #&  (df["rank"] == "12")
#get data
df[filt]

,meet,event,rank,name,age,team,seed,finals,seed_time
226,2021 PV Maryland State LC Championship - 6/4/2...,Girls 11-12 50 LC Meter Freestyle,36,"Kertajaya, Evelyn",12,RMSC!-PV,34.32,34.42,34.32
326,2021 PV Maryland State LC Championship - 6/4/2...,Girls 11-12 50 LC Meter Breaststroke,12,"Kertajaya, Evelyn",12,RMSC!-PV,43.68,42.75,43.68
352,2021 PV Maryland State LC Championship - 6/4/2...,Girls 11-12 100 LC Meter Breaststroke,16,"Kertajaya, Evelyn",12,RMSC!-PV,1:45.62,1:35.43,105.62
392,2021 PV Maryland State LC Championship - 6/4/2...,Girls 11-12 50 LC Meter Butterfly,23,"Kertajaya, Evelyn",12,RMSC!-PV,39.43,38.00,39.43


In [66]:
#this cell is cleaning up seed and finals time

#filter NT from final
filt_NT = (df["seed"] == "NT")
#update dq to 00.00
df["seed"][filt_NT] = "00.00"

#filter DQ from final
filt_DQ = (df["finals"] == "DQ")
#update dq to 00.00
df["finals"][filt_DQ] = "00.00"

<ipython-input-66-0421b20935eb>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["seed"][filt_NT] = "00.00"
<ipython-input-66-0421b20935eb>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["finals"][filt_DQ] = "00.00"


In [20]:
df["seed"].head()

0    33.04
1    32.66
2    34.25
3    33.27
4    34.62
Name: seed, dtype: object

In [41]:
#craete function to convert time string to float
def f_totime(i):
    #find minute indicator
    minute_ind = i.find(':')
    if (minute_ind != -1):
        i = float(i[0:minute_ind]) * 60 + float(i[minute_ind:].replace(":", ""))
        return i
    else:
        return i

# #testing fuction
# i = "10:29.05"

# result = f_totime(i)
# print(result)

In [67]:
#create new column from seed
df["seed_time"] = df.apply(lambda row: f_totime(row["seed"]),axis=1)
#create new column from final
df["finals_time"] = df.apply(lambda row: f_totime(row["finals"]),axis=1)

In [68]:
#change data type for seed time and final time
df["finals_time"] = df["finals_time"].astype(np.float16)

df["seed_time"] = df["seed_time"].astype(np.float16)

#create delta column
df["delta"] = df["finals_time"] - df["seed_time"]


In [72]:
#function converting delta gain/drop/same
def f_gain_drop(i):
    if i < 0:
        return "Gain"
    elif i > 0:
        return "Drop"
    else:
        return "Same"

In [73]:
#create new column
df["gain_drop_same"] = df.apply(lambda row: f_gain_drop(row["delta"]),axis=1)

In [74]:
#filtering examples
filt = (df["name"] == "Kertajaya, Evelyn") #&  (df["rank"] == "12")
#get data
df[filt]

,meet,event,rank,name,age,team,seed,finals,seed_time,finals_time,delta,gain_drop_same
226,2021 PV Maryland State LC Championship - 6/4/2...,Girls 11-12 50 LC Meter Freestyle,36,"Kertajaya, Evelyn",12,RMSC!-PV,34.32,34.42,34.3125,34.40625,0.09375,Drop
326,2021 PV Maryland State LC Championship - 6/4/2...,Girls 11-12 50 LC Meter Breaststroke,12,"Kertajaya, Evelyn",12,RMSC!-PV,43.68,42.75,43.6875,42.75000,-0.93750,Gain
352,2021 PV Maryland State LC Championship - 6/4/2...,Girls 11-12 100 LC Meter Breaststroke,16,"Kertajaya, Evelyn",12,RMSC!-PV,1:45.62,1:35.43,105.6250,95.43750,-10.18750,Gain
392,2021 PV Maryland State LC Championship - 6/4/2...,Girls 11-12 50 LC Meter Butterfly,23,"Kertajaya, Evelyn",12,RMSC!-PV,39.43,38.00,39.4375,38.00000,-1.43750,Gain
